In [3]:
import pandas as pd
import numpy as np
def get_mut(row):
    try:
        wt_aa = row["clean_mut_wt"][0]
        mut_aa = row["clean_mut_wt"][-1]
        pos = int(row["clean_mut_wt"][1:-1])
        seq = list(row["Sequence"])
        assert len(seq) > pos
        assert seq[pos-1] == wt_aa
        seq[pos-1] = mut_aa
        return "".join(seq)
    except AssertionError:
        return None
df = pd.read_csv("dataset/intenzy.csv")
df = df[df["substrate_kinetics"].apply(lambda x: ";" not in x)].reset_index(drop=True)
df["val"] = np.log10((df["kcat_mut"]/df["Km_mut"])/(df["kcat_wt"]/df["Km_wt"]))
df["p_kcat"] = np.log10((df["kcat_mut"]/df["kcat_wt"]))
df["p_km"] = np.log10((df["Km_mut"]/df["Km_wt"]))
seqs = pd.read_csv("dataset/idmapping.tsv", sep="\t")[["From", "Entry", "Sequence"]]
seqs.dropna(inplace=True)
df = pd.merge(df, seqs, left_on="uniprot", right_on="From", how="inner")
df.drop(columns=["From", "Entry"], inplace=True)
df["mut_seq"] = df.apply(get_mut, axis=1)
df.dropna(subset=["mut_seq"], inplace=True)
smiles_df = pd.read_csv("dataset/smiles.smi")
cleaned_df = pd.merge(df, smiles_df, left_on="substrate_kinetics", right_on="name")[["_id", "Sequence", "mut_seq", "smiles", "val"]]
cleaned_df.columns = ["id", "wt_seqs", "mut_seqs", "smiles", "val"]
cleaned_df.to_csv("dataset/train_data.csv", index=False)
cleaned_df

,id,wt_seqs,mut_seqs,smiles,val
0,63262552b980af2249bd882b,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CCCCCCCO,-1.238986
1,63262552b980af2249bd882f,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,C1=CC=C(C=C1)CO,-0.676442
2,63262552b980af2249bd8836,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISAGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,-0.268555
3,63262552b980af2249bd8835,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,MKAAVVEQFKEPLKIKEVEKPTISYGEVLVRIKACGVCHTDLHAAH...,C1=CC=C(C=C1)CO,0.006921
4,63262552b980af2249bd8830,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,MRAVRLVEIGKPLSLQEIGVPKPKGPQVLIKVEAAGVCHSDVHMRQ...,CC(C)C=O,-1.652125
...,...,...,...,...,...
2535,63262557b980af2249bd98a9,MESTLGWSVQDWLSFHSKSTPTKSLELLENLLKSQKPAPEDPAWIS...,MESTLGWSVQDWLSFHSKSTPTKSLELLENLLKSQKPAPEDPAWIS...,C(=O)(N)N,-4.211128
2536,63262557b980af2249bd98b1,MTENIHKHRILILDFGSQYTQLVARRVRELGVYCELWAWDVTEAQI...,MTENIHKHRILILDFGSQYTQLVARRVRELGVYCELWAWDVTEAQI...,C1=NC2=C(N1C3C(C(C(O3)COP(=O)(O)O)O)O)NC(=O)NC2=O,-0.060586
2537,63262557b980af2249bd98b0,MTENIHKHRILILDFGSQYTQLVARRVRELGVYCELWAWDVTEAQI...,MTENIHKHRILILDFGSQYTQLVARRVRELGVYCELWAWDVTEAQI...,C1=NC2=C(N1C3C(C(C(O3)COP(=O)(O)O)O)O)NC(=O)NC2=O,-0.461130
2538,63262557b980af2249bd97da,MWKLKIAEGGNDEYLYSTNNYVGRQTWVFDPQPPTPQELAQVQQAR...,MWKLKIAEGGNDEYLYSTNNYVGRQTWVFDPQPPTPQELAQVQQAR...,CC(=CCCC(=CCCC(=CCCC=C(C)CCC=C(C)CCC1C(O1)(C)C...,-4.958785
